# Unit 4: Programmieren Sie Ihren ersten Deep Reinforcement Learning Algorithmus mit PyTorch: Reinforce. Und teste seine Robustheit 💪.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail"/>


In diesem Notizbuch werden Sie Ihren ersten Deep Reinforcement Learning-Algorithmus von Grund auf programmieren: Reinforce (auch Monte Carlo Policy Gradient genannt).

Reinforce ist eine *Policy-basierte Methode*: ein Deep Reinforcement Learning-Algorithmus, der versucht, **die Policy direkt zu optimieren, ohne eine Action-Value-Funktion** zu verwenden.

Genauer gesagt ist Reinforce eine *Policy-Gradient-Methode*, eine Unterklasse von *Policy-basierten Methoden*, die darauf abzielt, **die Policy direkt zu optimieren, indem sie die Gewichte der optimalen Policy mithilfe des Gradientenanstiegs schätzt**.

Um ihre Robustheit zu testen, werden wir sie in 2 verschiedenen einfachen Umgebungen trainieren:
- Cartpole-v1
- Pixelcopter-Umgebung

⬇️ Hier ist ein Beispiel für das, was **am Ende dieses Notizbuchs erreicht wird** ⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Umgebungen"/>


### 🎮 Umgebungen:

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL-Library:

- Python
- PyTorch


Wir versuchen ständig, unsere Tutorials zu verbessern. **Wenn Sie also Fehler in diesem Notizbuch** finden, öffnen Sie bitte [einen Fehler im GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Ziele dieses Notizbuchs 🏆
Am Ende des Notizbuchs werden Sie:
- In der Lage sein, **einen Reinforce-Algorithmus mit PyTorch von Grund auf zu programmieren.**
- In der Lage sein, **die Robustheit deines Agenten unter Verwendung einfacher Umgebungen zu testen.**
- In der Lage sein, **Ihren trainierten Agenten auf den Hub** zu pushen, mit einer schönen Videowiedergabe und einem Bewertungsergebnis 🔥.

## Dieses Notizbuch stammt aus dem Deep Reinforcement Learning Kurs.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In diesem kostenlosen Kurs lernen Sie:

- 📖 Deep Reinforcement Learning in **Theorie und Praxis** studieren.
- 🧑‍💻 Lernen Sie, **berühmte Deep RL-Bibliotheken** wie Stable Baselines3, RL Baselines3 Zoo, CleanRL und Sample Factory 2.0 zu verwenden.
- 🤖 Trainieren Sie **Agenten in einzigartigen Umgebungen**.

Und mehr, siehe 📚 den Lehrplan 👉 https://simoninithomas.github.io/deep-rl-course

Vergessen Sie nicht, sich **<a href="http://eepurl.com/ic5ZUD">für den Kurs anzumelden</a>** (wir sammeln Ihre E-Mail, um Ihnen **die Links zu senden, wenn die einzelnen Einheiten veröffentlicht werden, und Sie über die Herausforderungen und Aktualisierungen zu informieren).**


Der beste Weg, um in Kontakt zu bleiben, ist, unserem Discord-Server beizutreten, um sich mit der Community und mit uns auszutauschen 👉🏻 https://discord.gg/ydHrjt3WP5

# Was sind die richtlinienbasierten Methoden?

Das Hauptziel des Reinforcement Learning ist es, **die optimale Strategie zu finden, die die erwartete kumulative Belohnung maximiert**.
Da das Reinforcement Learning auf der *Belohnungshypothese* basiert: **alle Ziele können als Maximierung der erwarteten kumulativen Belohnung beschrieben werden**.

Bei einem Fußballspiel zum Beispiel (bei dem man die Agenten in zwei Einheiten trainiert) ist das Ziel, das Spiel zu gewinnen. Wir können dieses Ziel beim Reinforcement Learning wie folgt beschreiben
**Maximierung der Anzahl der erzielten Tore** (wenn der Ball die Torlinie überquert) in die Fußballtore des Gegners. Und **Minimierung der Anzahl der Tore in den eigenen Fußballtoren**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/soccer.jpg" alt="Fußball" />

## Wertbasierte, richtlinienbasierte und akteurskritische Methoden

In der ersten Einheit haben wir zwei Methoden kennengelernt, um die optimale Policy zu finden (oder sich ihr in den meisten Fällen zu nähern).

- Bei *wertbasierten Methoden* lernen wir eine Wertfunktion.
  - Die Idee ist, dass eine optimale Wertfunktion zu einer optimalen Policy \\(\pi^{*}\) führt.
  - Unser Ziel ist es, **den Verlust zwischen dem vorhergesagten und dem Zielwert zu minimieren**, um die wahre Aktions-Wert-Funktion zu approximieren.
  - Wir haben eine Strategie, aber sie ist implizit, da sie **direkt aus der Wertfunktion** generiert wird. Beim Q-Learning haben wir zum Beispiel eine (epsilon-)gierige Strategie verwendet.

- Im Gegensatz dazu lernen wir bei *policy-basierten Methoden* direkt, \\(\pi^{*}\) zu approximieren, ohne eine Wertfunktion lernen zu müssen.
  - Die Idee ist **die Parametrisierung der Policy**. Wenn man beispielsweise ein neuronales Netz \\(\pi_\theta\\) verwendet, wird diese Strategie eine Wahrscheinlichkeitsverteilung über Aktionen ausgeben (stochastische Strategie).
  - <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/stochastic_policy.png" alt="stochastic policy" />
  - Unser Ziel ist dann **die Maximierung der Leistung der parametrisierten Policy unter Verwendung des Gradientenanstiegs**.
  - Zu diesem Zweck kontrollieren wir den Parameter \\(\theta\), der die Verteilung der Aktionen über einen Zustand beeinflussen wird.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/policy_based.png" alt="Policy based" />

- Nächstes Mal werden wir uns mit der *Actor-Critic*-Methode befassen, die eine Kombination aus wertbasierten und richtlinienbasierten Methoden darstellt.

Dank der Policybasierten Methoden können wir unsere Policy \\(\pi_\theta\\) direkt optimieren, um eine Wahrscheinlichkeitsverteilung über Aktionen \\(\pi_\theta(a|s)\\\) auszugeben, die zur besten kumulativen Rendite führt.
Dazu definieren wir eine Zielfunktion \\(J(\theta)\\), d.h. die erwartete kumulative Belohnung, und wir **möchten den Wert \\(\theta\\) finden, der diese Zielfunktion maximiert**.

## Der Unterschied zwischen richtlinienbasierten und richtliniengradientenbasierten Methoden

Policy-Gradient-Methoden, die wir in dieser Einheit untersuchen werden, sind eine Unterklasse der Policy-basierten Methoden. Bei richtlinienbasierten Methoden erfolgt die Optimierung die meiste Zeit *auf der Grundlage von Richtlinien*, da wir für jede Aktualisierung nur Daten (Trajektorien) verwenden, die **durch unsere letzte Version von** gesammelt wurden.

Der Unterschied zwischen diesen beiden Methoden **liegt darin, wie wir den Parameter** \\(\theta\\) optimieren:

- Bei *Policy-basierten Methoden* suchen wir direkt nach der optimalen Policy. Wir können den Parameter \\(\theta\\) **indirekt** optimieren, indem wir die lokale Approximation der Zielfunktion mit Techniken wie Hill Climbing, Simulated Annealing oder Evolutionsstrategien maximieren.
- Bei *Policy-Gradient-Methoden*, die eine Unterklasse der Policy-basierten Methoden sind, suchen wir direkt nach der optimalen Policy. Wir optimieren jedoch den Parameter \\(\theta\) **direkt**, indem wir den Gradientenanstieg auf die Leistung der Zielfunktion \\(J(\theta)\\) anwenden.

Bevor wir uns näher mit der Funktionsweise von Policy-Gradient-Methoden befassen (Zielfunktion, Policy-Gradient-Theorem, Gradientenanstieg usw.), wollen wir die Vor- und Nachteile von Policy-basierten Methoden untersuchen.

# Die Vor- und Nachteile der Policy-Gradient-Methoden

An dieser Stelle werden Sie vielleicht fragen: "Aber Deep Q-Learning ist hervorragend! Warum also Policy-Gradient-Methoden verwenden?". Um diese Frage zu beantworten, lassen Sie uns die **Vor- und Nachteile von Policy-Gradient-Methoden** untersuchen.

## Vorteile

Es gibt mehrere Vorteile gegenüber wertbasierten Methoden. Sehen wir uns einige von ihnen an:

### Die Einfachheit der Integration

Wir können die Policy direkt schätzen, ohne zusätzliche Daten (Aktionswerte) zu speichern.

### Policy-Gradient-Methoden können eine stochastische Policy lernen

Policy-Gradient-Methoden können **eine stochastische Policy erlernen, während Wertfunktionen dies nicht können**.

Dies hat zwei Konsequenzen:

1. Wir **müssen einen Kompromiss zwischen Exploration und Ausbeutung nicht von Hand implementieren**. Da wir eine Wahrscheinlichkeitsverteilung über Aktionen ausgeben, erkundet der Agent **den Zustandsraum, ohne immer dieselbe Flugbahn zu nehmen**.

2. Wir beseitigen auch das Problem des **perceptual aliasing**. Perceptual aliasing bedeutet, dass zwei Zustände gleich erscheinen (oder sind), aber unterschiedliche Aktionen erfordern.

Nehmen wir ein Beispiel: Wir haben einen intelligenten Staubsauger, dessen Ziel es ist, den Staub zu saugen und die Hamster nicht zu töten.

<figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/hamster1.jpg" alt="Hamster 1"/>
</figure>

Unser Staubsauger kann nur wahrnehmen, wo die Wände sind.

Das Problem ist, dass die **zwei roten (farbigen) Zustände Aliasing-Zustände sind, weil der Agent jeweils eine obere und untere Wand wahrnimmt**.

<figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/hamster2.jpg" alt="Hamster 1"/>
</figure>

Bei einer deterministischen Strategie wird der Agent entweder immer nach rechts gehen, wenn er sich im roten Bereich befindet, oder immer nach links. **In beiden Fällen bleibt unser Agent stecken und kann sich nicht aus dem Staub machen**.

Mit einem wertbasierten Verstärkungslernalgorithmus lernen wir eine **quasi-deterministische Strategie** ("gierige Epsilon-Strategie"). Folglich kann unser Agent **viel Zeit damit verbringen, den Staub zu finden**.

Andererseits bewegt sich eine optimale stochastische Strategie **in roten (farbigen) Zuständen zufällig nach links oder rechts**. Folglich **bleibt er nicht stecken und erreicht den Zielzustand mit einer hohen Wahrscheinlichkeit**.

<figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/hamster3.jpg" alt="Hamster 1"/>
</figure>

### Policy-Gradient-Methoden sind in hochdimensionalen Aktionsräumen und kontinuierlichen Aktionsräumen effektiver.

Das Problem mit Deep Q-learning ist, dass ihre **Vorhersagen in jedem Zeitschritt eine Punktzahl (maximale erwartete zukünftige Belohnung) für jede mögliche Aktion** zuweisen, wenn der aktuelle Zustand gegeben ist.

Was aber, wenn es unendlich viele Handlungsmöglichkeiten gibt?

Bei einem selbstfahrenden Auto zum Beispiel kann man in jedem Zustand eine (fast) unendliche Auswahl an Aktionen haben (das Lenkrad um 15°, 17,2°, 19,4° drehen, hupen usw.). **Wir müssen für jede mögliche Aktion einen Q-Wert ausgeben**! Und **die maximale Aktion einer kontinuierlichen Ausgabe zu wählen, ist selbst ein Optimierungsproblem**!

Mit Policy-Gradient-Methoden geben wir stattdessen eine **Wahrscheinlichkeitsverteilung über Aktionen** aus.

### Policy-Gradient-Methoden haben bessere Konvergenzeigenschaften

Bei wertbasierten Methoden verwenden wir einen aggressiven Operator, um **die Wertfunktion zu ändern: wir nehmen das Maximum über Q-Schätzungen**.
Folglich können sich die Aktionswahrscheinlichkeiten bei einer beliebig kleinen Änderung der geschätzten Aktionswerte drastisch ändern, wenn diese Änderung dazu führt, dass eine andere Aktion den maximalen Wert hat.

Wenn z. B. während des Trainings die beste Aktion links war (mit einem Q-Wert von 0,22) und im nächsten Trainingsschritt rechts (da der rechte Q-Wert 0,23 wird), haben wir die Strategie drastisch geändert, da die Strategie nun die meiste Zeit rechts statt links wählt.

Andererseits ändern sich bei Policy-Gradient-Methoden die stochastischen Handlungspräferenzen (Wahrscheinlichkeit, eine Handlung vorzunehmen) **mit der Zeit gleichmäßig**.

## Nachteile

Natürlich haben Policy-Gradient-Methoden auch einige Nachteile:

- **Häufig konvergiert die Policy-Gradient-Methode zu einem lokalen Maximum anstatt zu einem globalen Optimum**.
- Policy-Gradient-Methoden gehen langsamer vor, **Schritt für Schritt: es kann länger dauern, sie zu trainieren (ineffizient).**
- Policy-Gradient kann eine hohe Varianz haben. Wir werden in der Einheit zur Akteurskritik sehen, warum das so ist, und wie wir dieses Problem lösen können.

👉 Wenn Sie sich eingehender mit den Vor- und Nachteilen von Policy-Gradient-Methoden beschäftigen möchten, [können Sie sich dieses Video ansehen] (https://youtu.be/y3oqOjHilio).


# Vertiefung der Policy-Gradient-Methoden

## Das große Ganze im Blick

Wir haben gerade gelernt, dass Policy-Gradient-Methoden darauf abzielen, Parameter \\( \theta \\) zu finden, die **den erwarteten Ertrag** maximieren.

Die Idee ist, dass wir eine *parametrisierte stochastische Strategie* haben. In unserem Fall gibt ein neuronales Netz eine Wahrscheinlichkeitsverteilung über Aktionen aus. Die Wahrscheinlichkeit, jede Aktion durchzuführen, wird auch als *Aktionspräferenz* bezeichnet.

Nehmen wir das Beispiel von CartPole-v1:
- Als Eingabe haben wir einen Zustand.
- Als Ausgabe haben wir eine Wahrscheinlichkeitsverteilung über Aktionen in diesem Zustand.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/policy_based.png" alt="Policy based" />

Unser Ziel mit Policy-Gradient ist es, **die Wahrscheinlichkeitsverteilung von Aktionen** zu steuern, indem wir die Policy so abstimmen, dass **gute Aktionen (die den Ertrag maximieren) in der Zukunft häufiger abgerufen werden**.
Jedes Mal, wenn der Agent mit der Umwelt interagiert, verändern wir die Parameter so, dass gute Aktionen in der Zukunft häufiger durchgeführt werden.

Aber **wie sollen wir die Gewichte anhand des erwarteten Ertrags** optimieren?

Die Idee ist, dass wir den Agenten **während einer Episode interagieren lassen**. Wenn wir die Episode gewinnen, gehen wir davon aus, dass jede getätigte Aktion gut war und in Zukunft häufiger durchgeführt werden muss
da sie zum Sieg führen.

Wir wollen also für jedes Zustands-Aktions-Paar die \\(P(a|s)\\) erhöhen: die Wahrscheinlichkeit, diese Aktion in diesem Zustand durchzuführen. Oder verringern, wenn wir verloren haben.

Der Policy-Gradient-Algorithmus (vereinfacht) sieht wie folgt aus:
<figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_bigpicture.jpg" alt="Policy Gradient Big Picture"/>
</figure>

Jetzt, wo wir das große Bild haben, wollen wir tiefer in die Methoden des Policy-Gradienten eintauchen.

## Vertiefung der Policy-Gradient-Methoden

Wir haben unsere stochastische Policy \\(\pi\\), die einen Parameter \\(\theta\\) hat. Dieses \\(\pi\\) gibt bei einem Zustand **eine Wahrscheinlichkeitsverteilung von Aktionen** aus.

<figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/stochastic_policy.png" alt="Policy"/>
</figure>

Wobei \\(\pi_\theta(a_t|s_t)\\\) die Wahrscheinlichkeit ist, dass der Agent die Aktion \\(a_t\\) aus dem Zustand \\(s_t\\\) auswählt, wenn unsere Policy gegeben ist.

**Aber woher wissen wir, ob unsere Strategie gut ist?** Wir brauchen eine Möglichkeit, sie zu messen. Um das zu wissen, definieren wir eine Punktzahl/Zielfunktion namens \\(J(\theta)\\).

### Die Zielfunktion

Die *Zielfunktion* gibt uns die **Leistung des Agenten** bei einer Trajektorie (Zustandsaktionsfolge ohne Berücksichtigung der Belohnung (im Gegensatz zu einer Episode)), und sie gibt die *erwartete kumulative Belohnung* aus.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/objective.jpg" alt="Return"/>

Lassen Sie uns diese Formel etwas genauer erläutern:
- Die *erwartete Rendite* (auch *erwartete kumulative Belohnung* genannt), ist der gewichtete Durchschnitt (wobei die Gewichte durch \\(P(\tau;\theta)\\) aller möglichen Werte gegeben sind, die die Rendite \\(R(\tau)\\) annehmen kann).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/expected_reward.png" alt="Return"/>


- \\(R(\tau)\\) :  Die Rückkehr von einer beliebigen Flugbahn. Um diese Größe zur Berechnung der erwarteten Rendite zu verwenden, müssen wir sie mit der Wahrscheinlichkeit jeder möglichen Trajektorie multiplizieren.

- \\(P(\tau;\theta)\\) : Wahrscheinlichkeit jeder möglichen Trajektorie \\(\tau\\) (diese Wahrscheinlichkeit hängt von \\( \theta\\) ab, da sie die Policy definiert, die sie verwendet, um die Aktionen der Trajektorie auszuwählen, die einen Einfluss auf die besuchten Zustände hat).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/probability.png" alt="Probability"/>

- \\(J(\theta)\\) : Die erwartete Rendite berechnen wir, indem wir für alle Trajektorien die Wahrscheinlichkeit, diese Trajektorie zu nehmen, wenn \\(\theta \\) mit der Rendite dieser Trajektorie multipliziert wird.

Unser Ziel ist es dann, die erwartete kumulative Belohnung zu maximieren, indem wir das \\(\theta \\) finden, das die besten Aktionswahrscheinlichkeitsverteilungen ergibt:


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/max_objective.png" alt="Max objective"/>

## Gradientenanstieg und das Policy-Gradienten-Theorem

Policy-Gradient ist ein Optimierungsproblem: Wir wollen die Werte von \\(\theta\) finden, die unsere Zielfunktion \\(J(\theta)\\) maximieren, also müssen wir **Gradienten-Anstieg** verwenden. Dies ist die Umkehrung von *Gradient-Descent*, da sie die Richtung des steilsten Anstiegs von \\(J(\theta)\\) angibt.

(Wenn Sie eine Auffrischung des Unterschieds zwischen Gradientenabstieg und Gradientenaufstieg benötigen, lesen Sie [dies](https://www.baeldung.com/cs/gradient-descent-vs-ascent) und [dies](https://stats.stackexchange.com/questions/258721/gradient-ascent-vs-gradient-descent-in-logistic-regression)).

Unser Aktualisierungsschritt für den Gradienten-Abstieg ist:

\\( \theta \leftarrow \theta + \alpha * \nabla_\theta J(\theta) \\\)

Wir können diese Aktualisierung wiederholt anwenden, in der Hoffnung, dass \\\(\theta \\) zu dem Wert konvergiert, der \\\(J(\theta)\\\) maximiert.

Es gibt jedoch zwei Probleme bei der Berechnung der Ableitung von \\(J(\theta)\\):
1. Wir können die wahre Steigung der Zielfunktion nicht berechnen, da dies die Berechnung der Wahrscheinlichkeit jeder möglichen Flugbahn erfordert, was rechnerisch sehr aufwendig ist.
Wir wollen also eine **Gradientenschätzung mit einer stichprobenbasierten Schätzung berechnen (einige Trajektorien sammeln)**.

2. Wir haben ein weiteres Problem, das ich im nächsten optionalen Abschnitt erkläre. Um diese Zielfunktion zu differenzieren, müssen wir die Zustandsverteilung, die sogenannte Markov-Entscheidungsprozess-Dynamik, differenzieren. Diese ist mit der Umwelt verbunden. Sie gibt uns die Wahrscheinlichkeit, dass die Umgebung in den nächsten Zustand übergeht, wenn der aktuelle Zustand und die vom Agenten durchgeführte Aktion gegeben sind. Das Problem ist, dass wir sie nicht differenzieren können, weil wir sie möglicherweise nicht kennen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/probability.png" alt="Wahrscheinlichkeit"/>

Glücklicherweise werden wir eine Lösung namens Policy Gradient Theorem verwenden, die uns helfen wird, die Zielfunktion in eine differenzierbare Funktion umzuformulieren, die keine Differenzierung der Zustandsverteilung erfordert.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/policy_gradient_theorem.png" alt="Policy Gradient"/>

Wenn Sie verstehen wollen, wie wir diese Formel für die Annäherung an den Gradienten herleiten, lesen Sie den nächsten (optionalen) Abschnitt.

## Der Reinforce-Algorithmus (Monte Carlo Reinforce)

Der Reinforce-Algorithmus, auch Monte-Carlo-Policy-Gradient genannt, ist ein Policy-Gradient-Algorithmus, der **eine geschätzte Rendite aus einer ganzen Episode zur Aktualisierung des Policy-Parameters** verwendet:

In einer Schleife:
- Verwenden Sie die Richtlinie \\(\pi_\theta\\), um eine Episode zu sammeln \\(\tau\\)
- Verwenden Sie die Episode, um den Gradienten zu schätzen \\(\hat{g} = \nabla_\theta J(\theta)\\\)

 <figure class="image table text-center m-0 w-full">
  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/policy_gradient_one.png" alt="Policy Gradient"/>
</figure>

- Aktualisieren Sie die Gewichte der Richtlinie: \\(\theta \leftarrow \theta + \alpha \hat{g}\\)

Wir können diese Aktualisierung wie folgt interpretieren:
- \\(\nabla_\theta log \pi_\theta(a_t|s_t)\\\) ist die Richtung des **stärksten Anstiegs der (log)-Wahrscheinlichkeit** der Auswahl der Aktion at aus dem Zustand st.
Dies sagt uns, **wie wir die Gewichte der Policy ändern sollten**, wenn wir die logarithmische Wahrscheinlichkeit der Auswahl einer Aktion \\(a_t\\) im Zustand \\(s_t\\) erhöhen/verringern wollen.
- \\(R(\tau)\\): ist die Bewertungsfunktion:
  - Wenn die Rendite hoch ist, **erhöht sie die Wahrscheinlichkeiten** der Kombinationen (Zustand, Aktion).
  - Wenn die Rendite niedrig ist, **verringert sie die Wahrscheinlichkeiten** der (Zustand, Handlung) Kombinationen.


Wir können auch **mehrere Episoden (Trajektorien)** sammeln, um den Gradienten zu schätzen:
<figure class="image table text-center m-0 w-full">
 <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/policy_gradient_multiple.png" alt="Policy Gradient"/>
</figure>


# Glossar 

- **Deep Q-Learning:** Ein wertbasierter Deep Reinforcement Learning-Algorithmus, der ein tiefes neuronales Netz verwendet, um Q-Werte für Aktionen in einem bestimmten Zustand zu approximieren. Das Ziel von Deep Q-Learning ist es, die optimale Strategie zu finden, die die erwartete kumulative Belohnung durch Lernen der Aktionswerte maximiert.

- **Wertbasierte Methoden:** Reinforcement-Learning-Methoden, die eine Wertfunktion als Zwischenschritt zur Ermittlung einer optimalen Strategie schätzen.

- **Policybasierte Methoden:** Reinforcement Learning Methoden, die direkt lernen, die optimale Policy zu approximieren, ohne eine Wertfunktion zu lernen. In der Praxis geben sie eine Wahrscheinlichkeitsverteilung über Aktionen aus. 

    Die Vorteile der Verwendung von Policy-Gradienten-Methoden gegenüber wertbasierten Methoden sind unter anderem: 
    - Einfachheit der Integration: keine Notwendigkeit, Aktionswerte zu speichern;
    - die Fähigkeit, eine stochastische Strategie zu erlernen: der Agent erkundet den Zustandsraum, ohne immer dieselbe Flugbahn einzuschlagen, und vermeidet das Problem des Wahrnehmungs-Alias;
    - Effektivität in hochdimensionalen und kontinuierlichen Handlungsräumen; und
    - verbesserte Konvergenzeigenschaften.

- **Policy Gradient:** Eine Teilmenge der Policy-basierten Methoden, bei denen das Ziel darin besteht, die Leistung einer parametrisierten Policy mithilfe des Gradientenanstiegs zu maximieren. Das Ziel eines Policy-Gradienten ist es, die Wahrscheinlichkeitsverteilung von Aktionen zu kontrollieren, indem die Policy so eingestellt wird, dass gute Aktionen (die den Ertrag maximieren) in Zukunft häufiger abgerufen werden. 

- **Monte Carlo Reinforce:** Ein Policy-Gradient-Algorithmus, der eine geschätzte Rendite aus einer ganzen Episode verwendet, um den Policy-Parameter zu aktualisieren.

## Voraussetzungen 🏗️
Bevor Sie sich mit dem Notebook beschäftigen, müssen Sie:

🔲 📚 [Studieren Sie Policy Gradients durch Lesen von Unit 4](https://huggingface.co/deep-rl-course/unit4/introduction)

# Wir programmieren den Reinforce-Algorithmus von Grund auf neu 🔥.


Um diese praktische Übung für den Zertifizierungsprozess zu validieren, müssen Sie Ihre trainierten Modelle an den Hub übertragen.

- Erhalten Sie ein Ergebnis von >= 350 für "Cartpole-v1".
- Erhalte ein Ergebnis von >= 5 für "PixelCopter".

Um dein Ergebnis zu finden, gehe zum Leaderboard und suche dein Modell, **das Ergebnis = mean_reward - std of reward**. **Wenn du dein Modell nicht auf der Bestenliste siehst, gehe unten auf der Bestenlistenseite und klicke auf die Schaltfläche "Aktualisieren "**.

Weitere Informationen über den Zertifizierungsprozess finden Sie in diesem Abschnitt 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process


## Ein Ratschlag 💡
Es ist besser, dieses Colab in einer Kopie auf Ihrem Google Drive auszuführen, so dass Sie **bei Zeitüberschreitungen** immer noch das gespeicherte Notizbuch auf Ihrem Google Drive haben und nicht alles von Grund auf neu ausfüllen müssen.

Dazu kannst du entweder "Strg + S" oder "Datei > Kopie in Google Drive speichern" verwenden.

## Erstellen einer virtuellen Anzeige 🖥

Während der Arbeit mit dem Notebook müssen wir ein Wiederholungsvideo erstellen. Dazu benötigen wir mit colab **einen virtuellen Bildschirm, um die Umgebung zu rendern** (und somit die Bilder aufzunehmen).

Daher wird die folgende Zelle die Librairies installieren und einen virtuellen Bildschirm erstellen und starten 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Installieren Sie die Abhängigkeiten 🔽.
Der erste Schritt besteht darin, die Abhängigkeiten zu installieren. Wir werden mehrere davon installieren:

- `gym`
- `gym-games`: Zusätzliche Gym-Umgebungen, die mit PyGame erstellt wurden.
- `huggingface_hub`: 🤗 dient als zentraler Ort, an dem jeder Modelle und Datensätze teilen und erforschen kann. Es bietet Versionierung, Metriken, Visualisierungen und andere Funktionen, die eine einfache Zusammenarbeit mit anderen ermöglichen.

Sie fragen sich vielleicht, warum wir gym und nicht gymnasium, eine neuere Version von gym, installieren? **Weil die Gym-Spiele, die wir verwenden, noch nicht mit Gym aktualisiert sind**.

Die Unterschiede, die Sie hier finden werden:
- In `gym` haben wir kein `terminated` und `truncated` sondern nur `done`.
- In `gym` liefert die Verwendung von `env.step()` die Ergebnisse `state, reward, done, info`.

Mehr über die Unterschiede zwischen Gym und Gymnasium kannst du hier erfahren 👉 https://gymnasium.farama.org/content/migration-guide/


Hier können Sie alle verfügbaren Reinforce-Modelle sehen 👉 https://huggingface.co/models?other=reinforce

Und hier finden Sie alle Deep Reinforcement Learning-Modelle 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning


In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

## Importieren der Pakete 📦
Zusätzlich zum Import der installierten Bibliotheken, importieren wir auch:

- `imageio`: Eine Bibliothek, die uns helfen wird, ein Wiedergabevideo zu erzeugen



In [ ]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

## Prüfen, ob wir eine GPU haben

- Prüfen wir, ob wir eine GPU haben
- Wenn dies der Fall ist, sollten Sie `Gerät:cuda0` sehen

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

Wir sind nun bereit, unseren Reinforce-Algorithmus zu implementieren 🔥.

# Erster Agent: Playing CartPole-v1 🤖

## Erstellen Sie die CartPole-Umgebung und verstehen Sie, wie sie funktioniert.
### [Die Umgebung 🎮](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)


### Warum verwenden wir eine einfache Umgebung wie CartPole-v1?
Wie in [Tipps und Tricks zum Verstärkungslernen] (https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html) erläutert, müssen Sie bei der Neuimplementierung Ihres Agenten **sich vergewissern, dass er korrekt funktioniert, und mit einfachen Umgebungen Fehler finden, bevor Sie tiefer gehen**. Denn die Fehlersuche ist in einfachen Umgebungen viel einfacher.


> Versuchen Sie, bei Spielzeugproblemen ein "Lebenszeichen" zu geben.


> Validieren Sie die Implementierung, indem Sie sie auf immer schwierigeren Umgebungen laufen lassen (Sie können die Ergebnisse mit dem RL-Zoo vergleichen). Für diesen Schritt müssen Sie normalerweise eine Hyperparameter-Optimierung durchführen.
___
### Die CartPole-v1-Umgebung

> Eine Stange ist über ein unbetätigtes Gelenk an einem Wagen befestigt, der sich auf einer reibungsfreien Bahn bewegt. Das Pendel wird aufrecht auf dem Wagen platziert und das Ziel ist es, die Stange auszubalancieren, indem Kräfte in die linke und rechte Richtung auf den Wagen ausgeübt werden.



Wir beginnen also mit CartPole-v1. Das Ziel ist es, den Wagen nach links oder rechts zu schieben, **so dass der Pol im Gleichgewicht bleibt.**

Die Episode endet, wenn:
- der Winkel der Stange größer als ±12° ist
- Die Position des Wagens ist größer als ±2,4
- Die Episodenlänge ist größer als 500

Wir erhalten eine Belohnung 💰 von +1 für jeden Zeitschritt, den der Pol im Gleichgewicht bleibt.

In [ ]:
env_id = "CartPole-v1"
# Create the env
env = gym.make(env_id)

# Create the evaluation env
eval_env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

## Bauen wir die Reinforce-Architektur auf
Diese Implementierung basiert auf zwei Implementierungen:
- [PyTorch official Reinforcement Learning example](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)
- [Verbesserung der Integration durch Chris1nexus](https://github.com/huggingface/deep-rl-class/pull/95)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce"/>

Wir wollen also:
- Zwei vollständig verbundene Schichten (fc1 und fc2).
- Verwendung von ReLU als Aktivierungsfunktion von fc1
- Verwendung von Softmax zur Ausgabe einer Wahrscheinlichkeitsverteilung über Aktionen

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Create two fully connected layers



    def forward(self, x):
        # Define the forward pass
        # state goes to fc1 then we apply ReLU activation function

        # fc1 outputs goes to fc2

        # We output the softmax
    
    def act(self, state):
        """
        Given a state, take action
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

### Lösung

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m)
        return action.item(), m.log_prob(action)

Ich habe einen Fehler gemacht, können Sie erraten, wo?

- Um das herauszufinden, machen wir einen Vorwärtspass:

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
debug_policy.act(env.reset())

- Hier sehen wir, dass die Fehlermeldung `ValueError: Das Wertargument von log_prob muss ein Tensor sein".

- Das bedeutet, dass `action` in `m.log_prob(action)` ein Tensor sein muss **aber das ist er nicht.**

- Wissen Sie, warum? Prüfen Sie die Funktion act und versuchen Sie herauszufinden, warum sie nicht funktioniert.

Hinweis 💡: Irgendetwas ist in dieser Implementierung falsch. Erinnern Sie sich daran, dass wir mit der Funktion act **eine Aktion aus der Wahrscheinlichkeitsverteilung über Aktionen** ziehen wollen.


### (Real) Lösung

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

Durch die Verwendung von CartPole war es einfacher zu debuggen, da **wir wissen, dass der Fehler von unserer Integration kommt und nicht von unserer einfachen Umgebung**.

- Da **wir eine Aktion aus der Wahrscheinlichkeitsverteilung über Aktionen** auswählen wollen, können wir nicht `Aktion = np.argmax(m)` verwenden, da dies immer die Aktion mit der höchsten Wahrscheinlichkeit ausgibt.

- Wir müssen es durch `action = m.sample()` ersetzen, das eine Aktion aus der Wahrscheinlichkeitsverteilung P(.|s) auswählt.

### Bauen wir den Reinforce-Trainingsalgorithmus auf
Dies ist der Pseudocode des Reinforce-Algorithmus:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Policy gradient pseudocode"/>
  

- Bei der Berechnung der Belohnung Gt (Zeile 6) sehen wir, dass wir die Summe der Rabattierten Belohnungn **ausgehend vom Zeitschritt t** berechnen.

- Warum? Weil unsere Policy nur **Handlungen auf der Grundlage der Konsequenzen verstärken sollte**: Belohnungen, die vor einer Handlung erzielt wurden, sind also nutzlos (da sie nicht auf die Handlung zurückzuführen sind), **nur die Belohnungen, die nach der Handlung kommen, zählen**.

- Bevor Sie dies kodieren, sollten Sie diesen Abschnitt lesen [lassen Sie sich nicht von der Vergangenheit ablenken] (https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#don-t-let-the-past-distract-you), in dem erklärt wird, warum wir den Reward-to-Go-Gradienten verwenden.

Wir verwenden eine interessante Technik, die von [Chris1nexus](https://github.com/Chris1nexus) kodiert wurde, um **die Belohnung in jedem Zeitschritt effizient zu berechnen**. In den Kommentaren wird das Verfahren erläutert. Zögern Sie nicht, auch [die PR-Erklärung zu lesen](https://github.com/huggingface/deep-rl-class/pull/95)
Aber im Großen und Ganzen geht es darum, **die Belohnung in jedem Zeitschritt effizient zu berechnen**.

Die zweite Frage, die Sie sich stellen können, ist **Warum minimieren wir den Verlust**? Sie sprachen von Gradientenaufstieg und nicht von Gradientenabstieg?

- Wir wollen unsere Nutzenfunktion $J(\theta)$ maximieren, aber in PyTorch wie in Tensorflow ist es besser, eine Zielfunktion zu **minimieren**.
    - Nehmen wir also an, wir wollen Aktion 3 in einem bestimmten Zeitschritt verstärken. Vor dem Training dieser Aktion ist P 0.25.
    - Wir wollen also $\theta$ so verändern, dass $\pi_\theta(a_3|s; \theta) > 0.25$
    - Da sich alle P zu 1 summieren müssen, wird max $\pi_\theta(a_3|s; \theta)$ die **Minimierung der anderen Aktionswahrscheinlichkeit$ bewirken.
    - Wir sollten PyTorch also anweisen, $1 - \pi_\theta(a_3|s; \theta)$ zu minimieren.
    - Diese Verlustfunktion nähert sich 0, wenn $\pi_\theta(a_3|s; \theta)$ sich 1 nähert.
    - Wir ermutigen also den Gradienten zu maximal $\pi_\theta(a_3|s; \theta)$


In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = # TODO: reset the environment
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = # TODO get the action
            saved_log_probs.append(log_prob)
            state, reward, done, _ = # TODO: take an env step
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(    ) # TODO: complete here        
       
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

#### Lösung

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t) 
        n_steps = len(rewards) 
        # Compute the discounted returns at each timestep,
        # as 
        #      the sum of the gamma-discounted return at time t (G_t) + the reward at time t
        #
        # In O(N) time, where N is the number of time steps
        # (this definition of the discounted return G_t follows the definition of this quantity 
        # shown at page 44 of Sutton&Barto 2017 2nd draft)
        # G_t = r_(t+1) + r_(t+2) + ...
        
        # Given this formulation, the returns at each timestep t can be computed 
        # by re-using the computed future returns G_(t+1) to compute the current return G_t
        # G_t = r_(t+1) + gamma*G_(t+1)
        # G_(t-1) = r_t + gamma* G_t
        # (this follows a dynamic programming approach, with which we memorize solutions in order 
        # to avoid computing them multiple times)
        
        # This is correct since the above is equivalent to (see also page 46 of Sutton&Barto 2017 2nd draft)
        # G_(t-1) = r_t + gamma*r_(t+1) + gamma*gamma*r_(t+2) + ...
        
        
        ## Given the above, we calculate the returns at timestep t as: 
        #               gamma[t] * return[t] + reward[t]
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed 
        ## if we were to do it from first to last.
        
        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( gamma*disc_return_t + rewards[t]   )    
            
        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is 
        # added to the standard deviation of the returns to avoid numerical instabilities        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)
        
        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()
        
        # Line 8: PyTorch prefers gradient descent 
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
    return scores

## Train it
- Wir sind nun bereit, unseren Agenten zu trainieren.
- Aber zuerst definieren wir eine Variable, die alle Trainingshyperparameter enthält.
- Sie können die Trainingsparameter ändern (und sollten 😉 )

In [ ]:
cartpole_hyperparameters = {
    "h_size": 16,
    "n_training_episodes": 1000,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [ ]:
# Create policy and place it to the device
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [ ]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"], 
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"], 
                   100)

## Bewertungsmethode definieren 📝
- Hier definieren wir die Auswertungsmethode, mit der wir unseren Reinforce-Agenten testen werden.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param policy: The Reinforce agent
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward
        
      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Bewerten Sie unseren Agenten 📈

In [ ]:
evaluate_agent(eval_env, 
               cartpole_hyperparameters["max_t"], 
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

### Veröffentlichen Sie unser trainiertes Modell auf dem Hub 🔥
Da wir nun gesehen haben, dass wir nach dem Training gute Ergebnisse erzielt haben, können wir unser trainiertes Modell mit einer Zeile Code auf dem Hub 🤗 veröffentlichen.

Hier ist ein Beispiel für eine Model Card:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/modelcard.png"/>

### Push zum Hub
#### Diesen Code nicht verändern

In [ ]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

In [ ]:
def record_video(env, policy, out_directory, fps=30):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []  
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Take the action (index) that have the maximum expected future reward given that state
    action, _ = policy.act(state)
    state, reward, done, info = env.step(action) # We directly put next_state = state for recording logic
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id, 
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """
  Evaluate, Generate a video and Upload a model to Hugging Face Hub.
  This method does the complete pipeline:
  - It evaluates the model
  - It generates the model card
  - It generates a replay video of the agent
  - It pushes everything to the Hub

  :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
  :param model: the pytorch model we want to save
  :param hyperparameters: training hyperparameters
  :param eval_env: evaluation environment
  :param video_fps: how many frame per seconds to record our video replay 
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()
  
  # Step 1: Create the repo
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)
  
    # Step 2: Save the model
    torch.save(model, local_directory / "model.pt")

    # Step 3: Save the hyperparameters to JSON
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)
    
    # Step 4: Evaluate the model and build JSON
    mean_reward, std_reward = evaluate_agent(eval_env, 
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"], 
                                            model)
    # Get datetime
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"], 
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # Write a JSON file
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # Step 5: Create the model card
    env_name = hyperparameters["env_id"]
    
    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # Add metrics
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** Agent playing **{env_id}**
  This is a trained model of a **Reinforce** agent playing **{env_id}** .
  To learn to use this model and train yours check Unit 4 of the Deep Reinforcement Learning Course: https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 6: Record a video
    video_path =  local_directory / "replay.mp4"
    record_video(env, model, video_path, video_fps)

    # Step 7. Push everything to the Hub
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")

### .

Mit "push_to_hub" **werten Sie aus, zeichnen ein Replay auf, generieren eine Modellkarte Ihres Agenten und schieben sie an den Hub**.

This way:
- Sie können **unsere Arbeit vorführen** 🔥.
- Sie können **Ihren Agenten beim Spielen visualisieren** 👀
- Du kannst **einen Agenten mit der Community teilen, den andere benutzen können** 💾
- Sie können **auf eine Bestenliste 🏆 zugreifen, um zu sehen, wie gut Ihr Agent im Vergleich zu Ihren Klassenkameraden abschneidet** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard


Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">


In [ ]:
notebook_login()

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login` (oder `login`)

3️⃣ Wir sind jetzt bereit, unseren trainierten Agenten mit der Funktion `package_to_hub()` an den 🤗 Hub 🔥 zu übertragen.

In [ ]:
repo_id = "" #TODO Define your repo id {username/Reinforce-{model-id}}
push_to_hub(repo_id,
                cartpole_policy, # The model we want to save
                cartpole_hyperparameters, # Hyperparameters
                eval_env, # Evaluation environment
                video_fps=30
                )

Nachdem wir nun die Robustheit unserer Implementierung getestet haben, wollen wir eine komplexere Umgebung ausprobieren: PixelCopter 🚁.




## Zweiter Agent: PixelCopter 🚁.

### Studiere die PixelCopter-Umgebung 👀
- [Die Umgebungsdokumentation](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)


In [ ]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

Der Beobachtungsraum (7) 👀:
- Spieler-Y-Position
- Spieler-Geschwindigkeit
- Abstand des Spielers zum Boden
- Abstand des Spielers zur Decke
- nächster Block x Abstand zum Spieler
- nächste Blöcke obere y-Position
- nächste Blöcke untere y-Position

Der Aktionsraum(2) 🎮:
- Nach oben (Gaspedal drücken)
- Nichts tun (nicht auf den Beschleuniger drücken)

Die Belohnungsfunktion 💰:
- Für jeden vertikalen Block, den er durchläuft, erhält er eine positive Belohnung von +1. Jedes Mal, wenn ein Endzustand erreicht wird, erhält er eine negative Belohnung von -1.

### Definieren Sie die neue Richtlinie 🧠
- Wir brauchen ein tieferes neuronales Netz, da die Umgebung komplexer ist

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # Define the three layers here

    def forward(self, x):
        # Define the forward process here
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

#### Lösung

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size*2)
        self.fc3 = nn.Linear(h_size*2, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

### Definieren Sie die Hyperparameter ⚙️
- Denn diese Umgebung ist komplexer.
- Insbesondere für die versteckte Größe benötigen wir mehr Neuronen.

In [ ]:
pixelcopter_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 50000,
    "n_evaluation_episodes": 10,
    "max_t": 10000,
    "gamma": 0.99,
    "lr": 1e-4,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

### Train it
- Wir sind jetzt bereit, unseren Agenten zu trainieren 🔥.

In [ ]:
# Create policy and place it to the device
# torch.manual_seed(50)
pixelcopter_policy = Policy(pixelcopter_hyperparameters["state_space"], pixelcopter_hyperparameters["action_space"], pixelcopter_hyperparameters["h_size"]).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters["lr"])

In [ ]:
scores = reinforce(pixelcopter_policy,
                   pixelcopter_optimizer,
                   pixelcopter_hyperparameters["n_training_episodes"], 
                   pixelcopter_hyperparameters["max_t"],
                   pixelcopter_hyperparameters["gamma"], 
                   1000)

### Veröffentlichen Sie unser trainiertes Modell auf dem Hub 🔥.

In [ ]:
repo_id = "" #TODO Define your repo id {username/Reinforce-{model-id}}
push_to_hub(repo_id,
                pixelcopter_policy, # The model we want to save
                pixelcopter_hyperparameters, # Hyperparameters
                eval_env, # Evaluation environment
                video_fps=30
                )

## Einige zusätzliche Herausforderungen 🏆
Die beste Art zu lernen **ist, Dinge selbst auszuprobieren**! Wie Sie gesehen haben, ist der derzeitige Agent nicht besonders gut. Als ersten Vorschlag können Sie für mehr Schritte trainieren. Versuchen Sie aber auch, bessere Parameter zu finden.

In der [Rangliste] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) finden Sie Ihre Agenten. Können Sie sich an die Spitze setzen?

Hier sind einige Ideen, um dies zu erreichen:
* Trainiere mehr Schritte
* Probiere verschiedene Hyperparameter aus, indem du dir ansiehst, was deine Klassenkameraden gemacht haben 👉 https://huggingface.co/models?other=reinforce
* **Pushen Sie Ihr neu trainiertes Modell** auf dem Hub 🔥
* **Verbesserung der Implementierung für komplexere Umgebungen** (wie wäre es z.B., das Netzwerk in ein Convolutional Neural Network zu ändern, um
Frames als Beobachtung zu behandeln)?

________________________________________________________________________

**Glückwunsch zum Abschluss dieser Einheit**! Es gab eine Menge Informationen.
Und herzlichen Glückwunsch zum Abschluss des Tutorials. Du hast gerade deinen ersten Deep Reinforcement Learning-Agenten von Grund auf mit PyTorch programmiert und ihn im Hub geteilt 🥳.

Zögern Sie nicht, diese Einheit zu iterieren **durch Verbesserung der Implementierung für komplexere Umgebungen** (wie wäre es zum Beispiel, das Netzwerk in ein Convolutional Neural Network zu ändern, um
Frames als Beobachtung zu behandeln)?

In der nächsten Einheit **werden wir mehr über Unity MLAgents** lernen, indem wir Agenten in Unity-Umgebungen trainieren. Auf diese Weise werden Sie bereit sein, an den **AI vs. AI Herausforderungen teilzunehmen, bei denen Sie Ihre Agenten trainieren werden
gegen andere Agenten in einer Schneeballschlacht und einem Fußballspiel antreten**.

Klingt lustig? Bis zum nächsten Mal!

Zum Schluss würden wir gerne **hören, was Sie über den Kurs denken und wie wir ihn verbessern können**. Wenn Sie also ein Feedback haben, bitte 👉 [füllen Sie dieses Formular aus] (https://forms.gle/BzKXWzLAGZESGNaE9)

Wir sehen uns in Referat 5! 🔥

### Keep Learning, stay awesome 🤗

